# Calibration of a detector on a translation table

The aim of this document is to explain how to use `pyFAI.goniometer` for calibrating the position detector from the translation table encoders.

Those data have been acquired at ESRF-ID29 in summer 2013 on a Pilatus 6M using Ceria (CeO2) as calibrant.
Seven images have been acquired with the detector moved between 15 cm and 45 cm from the sample position.
A prior calibration has been performed using the *MX-calibrate* script from the pyFAI suite. 
The control points extracted during this initial calibration have been used as a starting point for this calibration.

The raw data files are available at:
http://www.silx.org/pub/pyFAI/gonio/MX-ceria/

In [ ]:
# Initialization of the plotting library for use in the Jupyter notebook
%pylab nbagg

In [ ]:
# Loading of a few libraries

import time
start_time =time.time()
import os
import glob
import fabio
import pyFAI
from  pyFAI.goniometer import GeometryTransformation, GoniometerRefinement, Goniometer
from pyFAI.gui import jupyter

In [ ]:
# Configure data location
DATA_DIRECTORY = "/workspace/valls/data/goniometer/MX-ceria"

In [ ]:
# Loading of the list of files, and display of the first one with its headers

image_files = glob.glob(DATA_DIRECTORY + "/*.cbf")
image_files.sort()
print("List of images: " + ", ".join(image_files) + "." + os.linesep)
fimg = fabio.open(image_files[0])

print("Image headers:")
for key, value in  fimg.header.items():
    print("%s: %s"%(key,value))
    
jupyter.display(fimg.data, label=fimg.filename)

In [ ]:
# Definition of the geometry translation function:

geotrans = GeometryTransformation(param_names = ["dist_offset", "dist_scale", 
                                                 "poni1", "poni2", "rot1","rot2"],
                                  dist_expr="pos * dist_scale + dist_offset", 
                                  poni1_expr="poni1",
                                  poni2_expr="poni2", 
                                  rot1_expr="rot1", 
                                  rot2_expr="rot2", 
                                  rot3_expr="0.0")


# Definition of the function reading the detector position from the header of the image.

def get_distance(header):
    """Takes the header of the CBF-file and returns the distance of the detector"""
    dist = 0
    for line in header.get("_array_data.header_contents","").split("\n"):
        words = line.split()
        if words[1] == "Detector_distance":
            dist = float(words[2])
            break
    return dist

print("Distance:",get_distance(fimg.header))

In [ ]:
# Definition of the detector, the calibrant and extraction of the wavelength used from the headers

pilatus = pyFAI.detector_factory("Pilatus6M")
CeO2 = pyFAI.calibrant.CALIBRANT_FACTORY("CeO2")
for line in fimg.header.get("_array_data.header_contents","").split("\n"):
    words = line.split()
    if words[1] == "Wavelength":
        wavelength = float(words[2])*1e-10
        break
print("Wavelength:", wavelength)
CeO2.wavelength = wavelength

In [ ]:
# Definition of the geometry refinement: the parameter order is the same as the param_names

param = {"dist_offset":0, 
         "dist_scale":1,
         "poni1":0.2, 
         "poni2":0.2, 
         "rot1":0,
         "rot2":0}

gonioref = GoniometerRefinement(param, #initial guess
                                pos_function=get_distance,
                                trans_function=geotrans,
                                detector=pilatus,
                                wavelength=wavelength)
print("Empty refinement object:")
print(gonioref)

In [ ]:
# Let's populate the goniometer refinement object with all control point files:

ponis = glob.glob(DATA_DIRECTORY + "/*.poni")
ponis.sort()
print(ponis)
for fn in ponis:
    base = os.path.splitext(fn)[0]
    fimg = fabio.open(base + ".cbf")
    gonioref.new_geometry(base, image=fimg.data, metadata=fimg.header, control_points=base+".npt",
                          geometry=fn, calibrant=CeO2)

print("Filled refinement object:")
print(gonioref)
print(os.linesep+"\tLabel \t Distance")
for k, v in gonioref.single_geometries.items():
    print(k,v.get_position())

In [ ]:
# Display all images with associated calibration:

for sg in gonioref.single_geometries.values():
    jupyter.display(sg=sg)

In [ ]:
# Initial refinement of the translation table model

gonioref.refine2()

In [ ]:
# Save the result of the fitting to a file and display the content of the JSON file:

gonioref.save("ID29.json")
with open("ID29.json") as fd:
    print(fd.read())

In [ ]:
# Restore the translation table setting from the file

transtable = Goniometer.sload("ID29.json")
print("Translation table: \n",transtable)

In [ ]:
# Create a multi-geometry object for all images in this set:

distances = [get_distance(fabio.open(fn).header) for fn in image_files]
print("Distances: ", distances)
multigeo = transtable.get_mg(distances)
multigeo.radial_range=(0, 65)
print(multigeo)

In [ ]:
# Integrate the set of images in a single run:

res = multigeo.integrate1d([fabio.open(fn).data for fn in image_files], 10000)

# Display the result using matplotlib
fig, ax = subplots()
ax.plot(*res)
ax.set_xlabel(res.unit.label)
ax.set_ylabel("Intensity")
ax.set_xlim(17, 22)
ax.set_title("Zoom on the two first rings")

Accoring to the provious image, peaks look double which indicates a bad modeling of the setup or a bad fitting.
As the fitting ended successfully, the bug is likely in the model: let's allow the PONI to move with the distance

In [ ]:
# Let's refine poni1 and poni2 also as function of the distance:

geotrans2 = GeometryTransformation(param_names = ["dist_offset", "dist_scale", 
                                                  "poni1_offset", "poni1_scale",
                                                  "poni2_offset", "poni2_scale",
                                                  "rot1","rot2"],
                         dist_expr="pos * dist_scale + dist_offset", 
                         poni1_expr="pos * poni1_scale + poni1_offset",
                         poni2_expr="pos * poni2_scale + poni2_offset", 
                         rot1_expr="rot1", 
                         rot2_expr="rot2", 
                         rot3_expr="0.0")

#initial guess from former parameter set
param2 = (gonioref.nt_param(*gonioref.param))._asdict()
param2["poni1_offset"] = 0
param2["poni2_offset"] = 0
param2["poni1_scale"] = 1
param2["poni2_scale"] = 1

gonioref2 = GoniometerRefinement(param2, 
                                 pos_function = get_distance,
                                 trans_function=geotrans2,
                                 detector=pilatus,
                                 wavelength=wavelength)
gonioref2.single_geometries = gonioref.single_geometries.copy()
print(gonioref2)


In [ ]:
# Refinement of the second model with all distances free

gonioref2.refine2()

In [ ]:
# Integration of all images with the second model

multigeo2 = gonioref2.get_mg(distances)
multigeo2.radial_range=(0, 65)
print(multigeo2)
res2 = multigeo2.integrate1d([fabio.open(fn).data for fn in image_files], 10000)

# Display the result, zooming on the two first rings
fig, ax = subplots()
ax.plot(*res)
ax.plot(*res, label="only distance free")
ax.plot(*res2, label="distance and PONI free")
ax.set_ylabel("Intensity")
ax.set_xlim(17, 22)
ax.set_title("Zoom on the two first rings")
ax.set_xlabel(res2.unit.label)
ax.legend()

In [ ]:
# Re-extract many more control points from images for a better fit

for sg in gonioref2.single_geometries.values():
    sg.extract_cp(pts_per_deg=3)
    jupyter.display(sg=sg)

In [ ]:
# Refine again the model

gonioref2.refine2()

# Build the MultiGeometry integrator object

multigeo3 = gonioref2.get_mg(distances)
multigeo3.radial_range=(0, 65)
print(multigeo3)

# Perform the azimuthal integration
res3 = multigeo3.integrate1d([fabio.open(fn).data for fn in image_files], 10000)

# Display the result
fig, ax = subplots()
ax.plot(*res, label="only distance free")
ax.plot(*res2, label="distance and PONI free")
ax.plot(*res2, linestyle="--", label="distance and PONI free, more points")
ax.set_xlabel(res2.unit.label)
ax.set_xlim(17, 22)
ax.legend()

This re-extraction of control point did not help to get a sharper diffraction profile. This step was un-necessary.

## Conclusion
This notebook exposes the how to calibrate a translation table for a moving detector. It allows to:
* Check the proper alignement of the table regarding the actual beam 
* Check the encoder's precision (usually good) and offsets (arbitrary)
* Perform azimuthal integration to retrieve powder diffraction patterns at any position of the detector.


In [ ]:
print("Total execution time: %.3fs"%(time.time() - start_time))